In [1]:
library(tidyverse)

s6566 <- read_csv("./1965-66.csv", show_col_types = FALSE)
s8081 <- read_csv("./1980-81.csv", show_col_types = FALSE)
s8182 <- read_csv("./1981-82.csv", show_col_types = FALSE)
s8283 <- read_csv("./1982-83.csv", show_col_types = FALSE)
s8384 <- read_csv("./1983-84.csv", show_col_types = FALSE)
s8485 <- read_csv("./1984-85.csv", show_col_types = FALSE)
s8586 <- read_csv("./1985-86.csv", show_col_types = FALSE)
s8687 <- read_csv("./1986-87.csv", show_col_types = FALSE)
s8788 <- read_csv("./1987-88.csv", show_col_types = FALSE)
s8889 <- read_csv("./1988-89.csv", show_col_types = FALSE)
s8990 <- read_csv("./1989-90.csv", show_col_types = FALSE)
s9091 <- read_csv("./1990-91.csv", show_col_types = FALSE)
s9192 <- read_csv("./1991-92.csv", show_col_types = FALSE)
s9293 <- read_csv("./1992-93.csv", show_col_types = FALSE)
s9394 <- read_csv("./1993-94.csv", show_col_types = FALSE)
s9495 <- read_csv("./1994-95.csv", show_col_types = FALSE)
s9596 <- read_csv("./1995-96.csv", show_col_types = FALSE)

results <- bind_rows(s6566, s8081, s8182, s8283, s8384, s8586, s8687, s8788,s8889, s8990, s9091, s9192, s9293, s9394, s9495, s9596) %>%
    group_by(
        season
    ) %>%
    mutate(
        game_no = row_number()
    ) %>%
    select(season, game_no, scorers)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.2     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.2     v tibble    3.2.1
v lubridate 1.9.2     v tidyr     1.3.0
v purrr     1.0.1     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
players <- read_csv("../player-stats/player_ssns.csv", show_col_types = FALSE) %>%
    select(
        season,
        surname,
        player_name,
        disam_name
    ) %>%
    unique()

In [4]:
# Pivot scorers to wide format
scorers <- separate_wider_delim(
    results,
    scorers,
    delim = ",",
    names_sep = "",
    too_few = "align_start"
) %>%
mutate(
    across(starts_with("scorers"), str_squish)
  ) %>%
  pivot_longer(
    starts_with("scorers"),
    values_to = "scorer"
  ) %>%
  filter(
    !is.na(scorer)
  ) %>%
mutate(
  scorer_name = case_when(
    str_detect(scorer, "\\s\\d+$") ~ str_replace_all(scorer, "(?:\\s\\d+)?$", ""),
    TRUE ~ scorer
  ),
  goals_scored = case_when(
    str_detect(scorer, "\\s\\d+$") ~ str_extract(scorer, "\\d+$") %>% as.numeric(),
    TRUE ~ 1
  )
 ) %>%
left_join(
  players,
  by = c("season" = "season", "scorer_name" = "disam_name")
) %>%
group_by(
  season,
  game_no
) %>%
mutate(
  goals_for = sum(goals_scored),
  player_name = case_when(
    scorer_name == "OG" ~ "OG",
    .default = player_name
  )
)

# Check for players with missing full names
scorers %>% ungroup() %>% filter(is.na(player_name)) %>%
select(season, scorer_name) %>%
  unique() %>% arrange(scorer_name)

season,scorer_name
<chr>,<chr>


In [5]:
# Import results data
results_df <- read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", show_col_types = FALSE) %>%
  rename(game_no = ssn_game_no) %>%
  select(season, game_no, game_date, opposition, venue, score, goals_for, goals_against, generic_comp, game_type, league_tier, manager)

# Check for games where no of goals scores differs from scorer data
results_df %>%
  inner_join(
    scorers,
    by = c("season", "game_no")
  ) %>%
  select(
    season, game_no, game_date, opposition, venue, score, goals_for.x, goals_for.y
  ) %>%
  filter(
    goals_for.x != goals_for.y
  ) %>%
  unique()

season,game_no,game_date,opposition,venue,score,goals_for.x,goals_for.y
<chr>,<dbl>,<date>,<chr>,<chr>,<chr>,<dbl>,<dbl>


In [6]:
# Import Soccerbase goal data
goals_2 <- read_csv("https://raw.githubusercontent.com/petebrown/update-player-stats/main/data/players_df.csv", show_col_type = FALSE) %>%
    select(game_date, player_name, pl_goals) %>%
    filter(pl_goals > 0) %>%
    left_join(results_df %>% select(game_date, season, game_no), by = "game_date") %>%
    rename(
        goals_scored = pl_goals
    ) %>%
    select(season, game_no, player_name, goals_scored) %>%
    arrange(season, game_no)

In [10]:
# Bind new dataframes with Soccerbase data
scorers_long <- scorers %>%
    select(
        season, game_no, player_name, goals_scored
    ) %>%
    bind_rows(goals_2)

write_csv(scorers_long, "../output/scorers-long.csv")

# Join scorers to results
all_scorers <- scorers_long %>%
  left_join(results_df, by = c("season", "game_no"))

head(all_scorers, 3)

season,game_no,player_name,goals_scored,game_date,opposition,venue,score,goals_for,goals_against,generic_comp,game_type,league_tier,manager
<chr>,<dbl>,<chr>,<dbl>,<date>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>
1965/66,1,Barry Dyson,2,1965-08-21,Halifax Town,A,2-2,2,2,Football League,League,4,Dave Russell
1965/66,2,Alan King,1,1965-08-23,Chester,H,1-0,1,0,Football League,League,4,Dave Russell
1965/66,3,Frank Twist,1,1965-08-27,Port Vale,H,1-0,1,0,Football League,League,4,Dave Russell


In [11]:
# Pivot to wide format for attachment to results
scorers_wide <- scorers %>%
    select(season, game_no, player_name, goals_scored) %>%
    mutate(
        scorer = paste(player_name, goals_scored, sep = " ")
    ) %>%
    group_by(season, game_no) %>%
    summarise(
        scorers = paste(scorer, collapse = ", "),
        goals_for = sum(goals_scored),
        .groups = "drop"
    ) %>%
    mutate(
        scorers = str_replace_all(scorers, " 1", "")
    ) %>%
    select(season, game_no, scorers)

head(scorers_wide, 3)

write_csv(scorers_wide, "../output/scorers_wide.csv")

season,game_no,scorers
<chr>,<int>,<chr>
1965/66,1,Barry Dyson 2
1965/66,2,Alan King
1965/66,3,Frank Twist


In [12]:
all_scorers %>%
    filter(
        game_type == "League"
    ) %>%
    group_by(
        season,
        player_name
    ) %>%
    summarise(
        goals_scored = sum(goals_scored),
        .groups = "drop"
    ) %>%
    ungroup() %>%
    group_by(
        season
    ) %>%
    mutate(
        pc_gls = round((goals_scored / sum(goals_scored)) * 100, 1)
    ) %>%
    arrange(
        desc(pc_gls),
        season,
        player_name
    ) %>%
    group_by(season) %>%
    slice_max(
        goals_scored,
        n = 3
    ) %>%
    ungroup() %>%
    arrange(
        desc(pc_gls), desc(goals_scored), season, player_name
    ) %>%
    head(10)

season,player_name,goals_scored,pc_gls
<chr>,<chr>,<dbl>,<dbl>
2018/19,James Norwood,29,46.8
1987/88,Ian Muir,27,44.3
1995/96,John Aldridge,27,42.2
1991/92,John Aldridge,22,39.3
2013/14,Ryan Lowe,19,37.3
1986/87,Ian Muir,20,37.0
1994/95,John Aldridge,24,35.8
1988/89,Ian Muir,21,33.9
2017/18,Andy Cook,26,33.8


In [13]:
all_scorers %>%
filter(
    game_type == "League",
    league_tier < 5    
) %>%
group_by(season, player_name) %>%
summarise(
    goals = sum(goals_scored),
    .groups = "drop"
) %>%
arrange(
    desc(goals),
    season,
    player_name
) %>%
filter(
    goals >= 20
) %>%
filter(
    !player_name %in% c("John Aldridge", "Ian Muir")
)

season,player_name,goals
<chr>,<chr>,<dbl>
1965/66,Barry Dyson,30
2018/19,James Norwood,29
2002/03,Simon Haworth,20


In [16]:
mans_10 <- results_df %>%
    group_by(manager) %>%
    summarise(
        games = n()
    ) %>%
    filter(games >= 10)

all_scorers %>%
    filter(
        game_type == "League",
        manager %in% mans_10$manager
    ) %>%
    group_by(
        manager,
        player_name
    ) %>%
    summarise(
        goals_scored = sum(goals_scored),
        .groups = "drop"
    ) %>%
    ungroup() %>%
    group_by(
        manager
    ) %>%
    mutate(
        pc_gls = round((goals_scored / sum(goals_scored)) * 100, 1)
    ) %>%
    arrange(
        desc(pc_gls),
        manager,
        player_name
    ) %>%
    filter(
        manager == "Ian Dawes"
    )

manager,player_name,goals_scored,pc_gls
<chr>,<chr>,<dbl>,<dbl>
Ian Dawes,James Vaughan,4,23.5
Ian Dawes,Harvey Saunders,3,17.6
Ian Dawes,Josh Hawkes,2,11.8
Ian Dawes,Otis Khan,2,11.8
Ian Dawes,Kaiyne Woolery,1,5.9
Ian Dawes,Kieron Morris,1,5.9
Ian Dawes,Paul Lewis,1,5.9
Ian Dawes,Peter Clarke,1,5.9
Ian Dawes,Rhys Hughes,1,5.9
